In [1]:
from utils import *

# Data preprocessing : 

**Remarks** :
- The score appearance, aroma, palate, taste, overall do **not** have the **same ranges** in ba and rb
    - BA ranges : all 5, with 0.25 precision
    - RB ranges (resp.) : 5,10,5,10,20 with 1 precision
- the **rating is a weighted average** of the 5 mentioned score above
    - BA rating = 0.06 * appearance + 0.24 * aroma + 0.10 * palate + 0.40 * taste + 0.20 * overall 
    - RB rating = 0.1 * appearance + 0.1 * aroma + 0.1 * palate + 0.1 * taste + 0.1 * overall
- **z-score is only computed** for data occuring **in matched beer data**
    - The z-score of a beer, is the z-score of with all the ratings of the same beer occuring the **same year**
- avg_computed may differ from avg, since ba and rb drops some ratings on some criterion

+ bros_score in the BA dataset is a score given by the founders of BA 
+ ba_score is the score of the beer by the BA community 

- For more information :
    - *Hint* : *Looking at the [dataset exploration notebook](1.exploration.ipynb) should help you discover the dataset, and also help you understand which transformation to apply on the data*

**Goal** : Preprocess the dataset
- Handle nan values
- Type conversion
- Transform numerical values

## RAM limitation

If there is to much data to load, you can limit the number of ratings to 'reduced' for each loading of ratings/reviews dataframe. It takes only the first 'reduced' ratings/reviews

Avoid loading data you won't work with

In [8]:
#REDUCED = None # No reduction
REDUCED = 1e5  # Approx 0.1% of the reviews

## Load data

In [3]:
beers_ba, breweries_ba, users_ba, ratings_ba, reviews_ba = load_data('ba', reduced=REDUCED)

In [4]:
beers_rb, breweries_rb, users_rb, ratings_rb, reviews_rb = load_data('rb', reduced=REDUCED)

In [5]:
beers_matched, breweries_matched, users_matched, ratings_matched, reviews_matched = load_data('matched', reduced=REDUCED)
(users_matched, users_approx_matched) = users_matched
(ratings_matched, ratings_ba_matched, ratings_rb_matched) = ratings_matched
(ratings_with_text_ba_matched, ratings_with_text_rb_matched) = reviews_matched

/home/aducret/Documents/EPFL/ADA_Course/ada-2023-project-dada/src/utils.py:64: DtypeWarning: Columns (0,1,2,3,4,5,8,10,11,12,13,15,16,17,18,19,20,23,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  beers_matched = pd.read_csv(DATASET_MATCHED_DIR+'beers.csv.zip', header=0, compression=COMPRESSION)
/home/aducret/Documents/EPFL/ADA_Course/ada-2023-project-dada/src/utils.py:68: DtypeWarning: Columns (0,1,2,3,5,7,8,9,10,11,13,17,18,19,20,22,24,25,26,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_matched = pd.read_csv(DATASET_MATCHED_DIR+'ratings.csv.zip', header=0, compression=COMPRESSION)


,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
23,142544,Régab,37262,Societe des Brasseries du Gabon (SOBRAGA),Euro Pale Lager,1,1,2.88,NaN,NaN,4.5,2.880000,NaN,0,NaN
24,19590,Barelegs Brew,10093,Strangford Lough Brewing Company Ltd,English Pale Ale,4,4,3.85,NaN,NaN,4.5,3.845000,NaN,0,NaN
25,19827,Legbiter,10093,Strangford Lough Brewing Company Ltd,English Pale Ale,75,59,3.45,80.0,80.0,4.8,3.439867,-0.649167,59,3.504068
26,20841,St. Patrick's Ale,10093,Strangford Lough Brewing Company Ltd,English Pale Ale,8,6,3.86,NaN,NaN,6.0,3.888750,0.093110,6,3.768333
27,20842,St. Patrick's Best,10093,Strangford Lough Brewing Company Ltd,English Bitter,64,48,3.56,82.0,90.0,4.2,3.556094,-0.478579,48,3.580417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280814,19149,Diamond Stout,885,Summit Station Restaurant & Brewery,Irish Dry Stout,3,3,3.83,NaN,NaN,6.5,3.830000,NaN,0,NaN
280816,19142,IPA,885,Summit Station Restaurant & Brewery,English India Pale Ale (IPA),2,2,3.24,NaN,NaN,5.6,3.240000,NaN,0,NaN
280817,19141,Irvington Pale Ale,885,Summit Station Restaurant & Brewery,American Pale Ale (APA),3,3,3.60,NaN,NaN,6.8,3.596667,NaN,0,NaN
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,NaN,NaN,5.0,2.713333,NaN,0,NaN


## RAM limitation

If there is to much data to load, you can limit the number of ratings to 'reduced' for each loading of ratings/reviews dataframe. It takes only the first 'reduced' ratings/reviews

Avoid loading data you won't work with